# General usage of GeoHexViz's builder module
This notebook provides some general usage of GeoHexViz's main module.

## Importing the right module
In order to start building visualizations, we must first import the module that provides the tools to build them.
The important module to note, is the `builder.py` module.
This module contains the `PlotBuilder` class that allows for the incremental building of a visualization.
Generally, an import should look as follows:

In [ ]:
from geohexviz.builder import PlotBuilder

## Using a PlotBuilder
Making a PlotBuilder object should look like something in the form:

In [ ]:
myBuilder = PlotBuilder()

The PlotBuilder class has many properties so be sure to read up on the in-depth documentation.
The PlotBuilder can be fully instantiated from the constructor, but this is not recommended as it can cause user-borne mistakes.

### Setting the required layer of the PlotBuilder
In order to display hexagonally binned data, the hexbin layer of the PlotBuilder must be set.
This is done by invoking the function `set_hexbin()` on it.
At any point, this layer can be set again.
The `set_hexbin()` function has many important parameters, most notably the `data`, `latitude_field`, `longitude_field`, `hex_resolution`, and `hexbin_info` parameters.
The `data` parameter tells GeoHexViz where to obtain the input data from.
It can be a DataFrame, GeoDataFrame, or file path.
The parameters `latitue_field` and `longitude_field` determine what columns GeoHexViz parses for latitude and longitude values.
These parameters can be ignored when the columns containing latitudes and longitudes are given a distinguishable title, or when a GeoDataFrame with a filled geometry column is passed.
The `hex_resolution` parameter tells GeoHexViz what size of hexagon to use for the bins.
This value must be in the range 0 to 15 (0 being largest, 15 being smallest), and is defined by Uber H3.
Finally, the `hexbin_info` should itself contain two parameters that specify how the display value of each hexagon should be determined.
These parameters are `binning_field` which specified what column should be used, and `binning_fn` which specifies how the display value be calculated (what function to use on the grouped data).
An example of setting the hexbin layer is given below.

In [ ]:
# first make a DataFrame containing random data
from pandas import DataFrame
import random

df = DataFrame({
    'latitude': random.sample(range(-90, 90), 50),
    'longitude': random.sample(range(-180, 180), 50),
    'value': random.sample(range(0, 100), 50)
})

In [ ]:
df

In [ ]:
# now set te hexbin layer
myBuilder.set_hexbin(
    data=df,                      # recall that there are a variety of inputs for this parameter
    latitude_field='latitude',    # not necessary (distinguishable name)
    longitude_field='longitude'   # not necessary (distinguishable name)
    hex_resolution=3,             # typically resolution 3 or 4 is recommended
    hexbin_info=dict(
        binning_field='value',    # display value is based on the value column
        binning_fn='avg'          # display value is the average of the value column; per hex
    )
)

Note that the parameters for other layer types are very similar to the hexbin layer.

### Adding optional layers to the PlotBuilder
There are a variety of optional layers that can be added to the PlotBuilder.
These layer types include: region, outline, grid, and point.
Each layer type has its own purpose.

#### Adding a region layer
The purpose of a region-type layer is to highlight areas of interest.
The manifest visually as a solid-coloured polygon with a defined outline.
A region-type layer is usally defined by a GIS readable file, GeoDataFrame, country name, or continent name.
An example of adding a region-type layer is given below.

In [ ]:
myBuilder.add_region(
    name="sample_region",     # every optional layer-type needs a name
    data="CANADA"             # country name
)

With these two parameters alone, the region layer is defined.
It is worth noting that the `data` property of a region layer can not be in any form that contains anything other than polygon-like geometry.

#### Adding a outline layer
The purpose of a outline-type layer also is to highlight areas of interest. The manifest visually as a polygon with a defined outline. A outline-type layer is usally defined by a GIS readable file, GeoDataFrame, country name, continent name, or a file that contains coordinates. An example of adding a outline-type layer is given below.

In [ ]:
myBuilder.add_outline(
    name="sample_outline",
    data="UNITED STATES OF AMERICA"
)

#### Adding a grid layer
The purpose of a grid-type layer is to extend hexagonal tiling and help the hexbin layer form a continuous grid.
A grid-type layer is usually defined with a GIS readable file, GeoDataFrame, country name, or continent name.
They manifest visually as empty hexagons with defined outlines.
These layers require very similar properties to the hexbin layer.
An example of adding a grid-type layer is given below.

In [ ]:
# define a dataframe for a grid layer
grdf = DataFrame({
    'latitude': random.sample(range(-90, 90), 50),
    'atm_long_tt': random.sample(range(-180, 180), 50)
})

In [ ]:
grdf

In [ ]:
# add a grid layer
myBuilder.add_grid(
    name="sample_grid1",            # still requires a name
    data=grdf,
    latitude_field="latitude",      # still not necessary (distinguishable name)
    longitude_field="atm_long_tt",  # required (not a distinguishable name)
    hex_resolution=3                # if not specified, it copies the resolution of the hexbin layer
)

In [ ]:
# add another grid layer
myBuilder.add_grid(
    name="sample_grid2",
    data="CANADA"
)

#### Adding a point layer
The purpose of a point-type layer is to provide a way of visualizing scatter data over the hexbin layer.
A point-type layer is usually defined with a GIS readable file, DataFrame, or GeoDataFrame.
An example of adding a point-type layer is given below.

In [ ]:
# define a dataframe for a point layer
labels = [random.choice(["no", "yes", "maybe"]) for i in range(0, 50)]   # random list of labels for each point
ptdf = DataFrame({
    'latitude': random.sample(range(-90, 90), 50),
    'longitude': random.sample(range(-180, 180), 50),
    'text': labels
})

In [ ]:
ptdf

In [ ]:
# add a point layer
myBuilder.add_point(
    name="sample_point",            # still requires a name
    data=ptdf,
    text_field='text'               # not required, displays the text column over each point
)                                   # latitude field, longitude field not required (distinguishable name)

### Altering the PlotBuilder layers
There are functions that the user can invoke to alter the layers in the plot.
This section will highlight three of these functions.

#### Clipping data
The function `clip_layers` will clip the geometries of layer(s) to another layer(s).
It has two main parameters being `clip` which specifies what layer(s) will be clipped, and `to` which specifies what layers will act as the boundary of the clip.
An example of this function being used is given below.

In [ ]:
myBuilder.clip_layers(clip="hexbin", to="regions")

Running this clip the geometries of the hexbin layer to the geometries of the region layers within the PlotBuilder.

#### Logarithmic scale
The function `logify_scale` will make the plot use a logarithmic scale.
It has many parameters for specialized use but usually it does not require them.

In [ ]:
myBuilder.logify_scale(exp_type="^") # exp_type determines what type of exponent appears in the colour bar's numbers

#### Adjust focus
The function `adjust_focus` adjusts the plot such that it focuses on the geometries of the given layer.
For this reason, it takes one main parameter called `on` which determines which layer(s) to focus on.

In [ ]:
myBuilder.adjust_focus(on="regions") # focuses on all region-type layers within the PlotBuilder

### Visualization output
Before obtaining output, the visualization must first be built.
This can be done via invoking the `finalize` function:

In [ ]:
myBuilder.finalize()

Now the visualization can be displayed in the browser via invoking the `display` function.
In the browser, the visualization can be interacted with.

In [ ]:
myBuilder.display()

The visualization can also be output to a file by using the `output` function.
This function has only one required parameter which is `filepath`.

In [ ]:
myBuilder.output(filepath="yay.jpg")

Note that when in PDF format, and if the proper libraries are installed, the output can be cropped via the `crop_output` parameter.

In [ ]:
myBuilder.output(filepath="yay.pdf", crop_output=True)